In [1]:
import praw #Python Reddit API wrapper
import pandas as pd
from praw.models import MoreComments
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import FreqDist
import datetime as dt
import re # Remove urls
import emoji
import spacy
import en_core_web_sm

In [4]:
# Create a reddit connection with reddit API information
# Create praw.Reddit object with with reddit OAuth creds
# Reddit application creds created at https://www.reddit.com/prefs/apps
reddit = praw.Reddit(
            client_id= PRAWConfig.REDDIT_CLIENT_ID,
            client_secret= PRAWConfig.REDDIT_CLIENT_SECRET,
            user_agent= PRAWConfig.REDDIT_USER_AGENT)

### Submission Object

In [8]:
# Obtain title,'Submission ID' and date of the 20 'hottest' submissions in r/WallStreetbets that mentions '$AMC'
subreddit = reddit.subreddit('WallStreetBets')
ticker = "AMC" 

def get_date(date):
    return dt.datetime.fromtimestamp(date)

for submission in subreddit.search(ticker, sort='hot',limit=30):
        if submission.domain != "self.wallstreetbets":
            continue
        print(submission.title)
        print('Submission ID =', submission.id, '\n')
        print('date =', get_date(submission.created_utc) , '\n')
    

Daily Popular Tickers Thread for July 16, 2021 - AMC | GME | AAPL
Submission ID = oldtp9 

date = 2021-07-16 11:00:58 

Daily Popular Tickers Thread for July 15, 2021 - AMC | GME | AAPL
Submission ID = okpqv5 

date = 2021-07-15 11:00:58 

For AMC the Hedgefunds are not the real predators and have been waiting to post this
Submission ID = okj61o 

date = 2021-07-15 03:10:20 

Daily Option Statistics for AMC, GME, AAPL, SPCE, CLOV, WISH, TLRY, CLNE, PLTR and others
Submission ID = okcehp 

date = 2021-07-14 21:04:06 

AMC Options Analysis - Open Interest ITM - Puts = $1.55B Calls = $0.65B
Submission ID = okk9w1 

date = 2021-07-15 04:18:26 

Ride the Magic ShortBus TA DD - SPY, AMC, SPCE, and AAPL - 7/16
Submission ID = okcjtr 

date = 2021-07-14 21:11:07 

Daily Popular Tickers Thread for July 14, 2021 - AMC | CLOV | NOK
Submission ID = ok16jr 

date = 2021-07-14 11:00:59 

Current State of $GME and $AMC - Unbiased Technical Analysis
Submission ID = ojpqu3 

date = 2021-07-13 22:43:35 

In [15]:
# define a submission object
Post1 = reddit.submission(id='oldtp9')

In [20]:
# store all comments scraped from submission object in a list 
Comments_All = []
stock = "AMC"
Post1.comments.replace_more(limit=None)
for comments in Post1.comments.list():
    if any(keyword in comments.body for keyword in stock):     
        Comments_All.append(comments.body)

print(Comments_All, '\n')
print('Total Comments Scraped = ', (len(Comments_All)))


['AMC do that thing I like..', 'AAPL? Alright this thread has lost it', 'AMC drop looks as natural as a Kardashian bone structure', "Fuck, should've bought more GME at ~$150.", 'GME GOING TO THE OUTER BANKS OF THE STRATOSPHERE!! (allright its up +4.15%)', 'This is going to be a glorious $GMErection', 'GME holding strong for now, nice to see', 'Fly GME Fly', 'Come on GME, do that thing again!', 'Bought 5 more GMEs this am:)  continuing to averaging down. Easy money.', 'Poor AAPL, forced to spend the day on the short bus with the special kids again', 'Sold NVDA for GME!', 'Any green don’t matter until 9', 'Apple was a 30 year pump and dump 😢', 'Love my AAPL a day!!', 'It makes sense for AMC to gain on a Friday.  On Monday they will release their weekend box office numbers, and of course they will break all records for a the past 2 years...to get another great gain on Monday.', 'Guys $GME is pretty affordable now, I’m buying more at opening.', 'Here comes the Friday dump. Apes, let’s act 

### Preprocess The Comments

In [21]:
# convert to a string object
List1 = Comments_All
List1 = [str(i) for i in List1]
string_raw = ' , '.join(List1) 
string_raw

'AMC do that thing I like.. , AAPL? Alright this thread has lost it , AMC drop looks as natural as a Kardashian bone structure , Fuck, should\'ve bought more GME at ~$150. , GME GOING TO THE OUTER BANKS OF THE STRATOSPHERE!! (allright its up +4.15%) , This is going to be a glorious $GMErection , GME holding strong for now, nice to see , Fly GME Fly , Come on GME, do that thing again! , Bought 5 more GMEs this am:)  continuing to averaging down. Easy money. , Poor AAPL, forced to spend the day on the short bus with the special kids again , Sold NVDA for GME! , Any green don’t matter until 9 , Apple was a 30 year pump and dump 😢 , Love my AAPL a day!! , It makes sense for AMC to gain on a Friday.  On Monday they will release their weekend box office numbers, and of course they will break all records for a the past 2 years...to get another great gain on Monday. , Guys $GME is pretty affordable now, I’m buying more at opening. , Here comes the Friday dump. Apes, let’s act like we’ve been h

In [22]:
# remove emojis
string_noemoji = emoji.get_emoji_regexp().sub(u'', string_raw)
string_noemoji

'AMC do that thing I like.. , AAPL? Alright this thread has lost it , AMC drop looks as natural as a Kardashian bone structure , Fuck, should\'ve bought more GME at ~$150. , GME GOING TO THE OUTER BANKS OF THE STRATOSPHERE!! (allright its up +4.15%) , This is going to be a glorious $GMErection , GME holding strong for now, nice to see , Fly GME Fly , Come on GME, do that thing again! , Bought 5 more GMEs this am:)  continuing to averaging down. Easy money. , Poor AAPL, forced to spend the day on the short bus with the special kids again , Sold NVDA for GME! , Any green don’t matter until 9 , Apple was a 30 year pump and dump  , Love my AAPL a day!! , It makes sense for AMC to gain on a Friday.  On Monday they will release their weekend box office numbers, and of course they will break all records for a the past 2 years...to get another great gain on Monday. , Guys $GME is pretty affordable now, I’m buying more at opening. , Here comes the Friday dump. Apes, let’s act like we’ve been he

In [23]:
# tokenize and clean strings
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|http\S+')
tokenized_string = tokenizer.tokenize(string_noemoji)
tokenized_string

['AMC',
 'do',
 'that',
 'thing',
 'I',
 'like',
 'AAPL',
 'Alright',
 'this',
 'thread',
 'has',
 'lost',
 'it',
 'AMC',
 'drop',
 'looks',
 'as',
 'natural',
 'as',
 'a',
 'Kardashian',
 'bone',
 'structure',
 'Fuck',
 'should',
 've',
 'bought',
 'more',
 'GME',
 'at',
 '$150.',
 'GME',
 'GOING',
 'TO',
 'THE',
 'OUTER',
 'BANKS',
 'OF',
 'THE',
 'STRATOSPHERE',
 'allright',
 'its',
 'up',
 '4',
 '15',
 'This',
 'is',
 'going',
 'to',
 'be',
 'a',
 'glorious',
 'GMErection',
 'GME',
 'holding',
 'strong',
 'for',
 'now',
 'nice',
 'to',
 'see',
 'Fly',
 'GME',
 'Fly',
 'Come',
 'on',
 'GME',
 'do',
 'that',
 'thing',
 'again',
 'Bought',
 '5',
 'more',
 'GMEs',
 'this',
 'am',
 'continuing',
 'to',
 'averaging',
 'down',
 'Easy',
 'money',
 'Poor',
 'AAPL',
 'forced',
 'to',
 'spend',
 'the',
 'day',
 'on',
 'the',
 'short',
 'bus',
 'with',
 'the',
 'special',
 'kids',
 'again',
 'Sold',
 'NVDA',
 'for',
 'GME',
 'Any',
 'green',
 'don',
 't',
 'matter',
 'until',
 '9',
 'Apple',
 

In [24]:
# convert Tokens into lowercase letters
lc_tokenized_string = [word.lower() for word in tokenized_string]
lc_tokenized_string

['amc',
 'do',
 'that',
 'thing',
 'i',
 'like',
 'aapl',
 'alright',
 'this',
 'thread',
 'has',
 'lost',
 'it',
 'amc',
 'drop',
 'looks',
 'as',
 'natural',
 'as',
 'a',
 'kardashian',
 'bone',
 'structure',
 'fuck',
 'should',
 've',
 'bought',
 'more',
 'gme',
 'at',
 '$150.',
 'gme',
 'going',
 'to',
 'the',
 'outer',
 'banks',
 'of',
 'the',
 'stratosphere',
 'allright',
 'its',
 'up',
 '4',
 '15',
 'this',
 'is',
 'going',
 'to',
 'be',
 'a',
 'glorious',
 'gmerection',
 'gme',
 'holding',
 'strong',
 'for',
 'now',
 'nice',
 'to',
 'see',
 'fly',
 'gme',
 'fly',
 'come',
 'on',
 'gme',
 'do',
 'that',
 'thing',
 'again',
 'bought',
 '5',
 'more',
 'gmes',
 'this',
 'am',
 'continuing',
 'to',
 'averaging',
 'down',
 'easy',
 'money',
 'poor',
 'aapl',
 'forced',
 'to',
 'spend',
 'the',
 'day',
 'on',
 'the',
 'short',
 'bus',
 'with',
 'the',
 'special',
 'kids',
 'again',
 'sold',
 'nvda',
 'for',
 'gme',
 'any',
 'green',
 'don',
 't',
 'matter',
 'until',
 '9',
 'apple',
 

In [26]:
# remove stopwords *** stopwords are words that do not add much information to a sentence
spacy_nlp = en_core_web_sm.load()
all_sw = spacy_nlp.Defaults.stop_words

text = lc_tokenized_string
tokens_wosm = [word for word in text if not word in all_sw]

tokens_wosm



['amc',
 'thing',
 'like',
 'aapl',
 'alright',
 'thread',
 'lost',
 'amc',
 'drop',
 'looks',
 'natural',
 'kardashian',
 'bone',
 'structure',
 'fuck',
 've',
 'bought',
 'gme',
 '$150.',
 'gme',
 'going',
 'outer',
 'banks',
 'stratosphere',
 'allright',
 '4',
 '15',
 'going',
 'glorious',
 'gmerection',
 'gme',
 'holding',
 'strong',
 'nice',
 'fly',
 'gme',
 'fly',
 'come',
 'gme',
 'thing',
 'bought',
 '5',
 'gmes',
 'continuing',
 'averaging',
 'easy',
 'money',
 'poor',
 'aapl',
 'forced',
 'spend',
 'day',
 'short',
 'bus',
 'special',
 'kids',
 'sold',
 'nvda',
 'gme',
 'green',
 'don',
 't',
 'matter',
 '9',
 'apple',
 '30',
 'year',
 'pump',
 'dump',
 'love',
 'aapl',
 'day',
 'makes',
 'sense',
 'amc',
 'gain',
 'friday',
 'monday',
 'release',
 'weekend',
 'box',
 'office',
 'numbers',
 'course',
 'break',
 'records',
 'past',
 '2',
 'years',
 'great',
 'gain',
 'monday',
 'guys',
 'gme',
 'pretty',
 'affordable',
 'm',
 'buying',
 'opening',
 'comes',
 'friday',
 'dump',

In [27]:
cleaned_comments = tokens_wosm

### Pre-Processing vs. Post-Processing

In [28]:
print("Original length of words = ", (len(string_raw)))
print("Number of words after removing emojis = ", (len(string_noemoji)))
print("Number of words after removing tokenizing and cleaning = ", (len(tokenized_string)))
print("Number of words after removing stop words = ", (len(tokens_wosm)))

Original length of words =  34800
Number of words after removing emojis =  34446
Number of words after removing tokenizing and cleaning =  6602
Number of words after removing stop words =  3378


In [30]:
import pickle
with open('AMC.pkl', 'wb') as fp:
    pickle.dump(cleaned_comments, fp) 